
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session                                                                                                  |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X                                                                            |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0)                                |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |   Changes the session type to Glue ETL.                                                                                                                   |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer                       |

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.35 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::601591369946:role/Kshitij_Glue_Service_and_S3
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: dee30aae-2bfb-4dac-b067-e30c1e7734f5
Applying the following default arguments:
--glue_kernel_version 0.35
--enable-glue-datacatalog true
Waiting for session dee30aae-2bfb-4dac-b067-e30c1e7734f5 to get into ready status...
Session dee30aae-2bfb-4dac-b067-e30c1e7734f5 has been created




In [1]:
business_df = spark.read.parquet("s3://yelp-dataset-kshitij/output/yelp_academic_dataset_business/*.parquet")
tip_df = spark.read.parquet("s3://yelp-dataset-kshitij/output/yelp_academic_dataset_tip/*.parquet")
user_df = spark.read.parquet("s3://yelp-dataset-kshitij/output/yelp_academic_dataset_user/*.parquet")
review_df = spark.read.parquet("s3://yelp-dataset-kshitij/output/yelp_academic_dataset_review/*.parquet")

In [2]:
print("Total records in business file is : ",business_df.count())
print("Total records in tip file is : ",tip_df.count())
print("Total records in user file is : ",user_df.count())
print("Total records in review file is : ",review_df.count())

Total records in business file is :  42153
Total records in tip file is :  403210
Total records in user file is :  252898
Total records in review file is :  1125458


In [3]:
import pyspark.sql.functions as f
tip_df = tip_df.withColumn("tip_year",f.year(f.to_date(f.col("date"))))
tip_df = tip_df.withColumn("tip_month",f.month(f.to_date(f.col("date"))))
tip_df.show()

+--------------------+----------+-----+--------------------+----+--------------------+--------+---------+
|         business_id|      date|likes|                text|type|             user_id|tip_year|tip_month|
+--------------------+----------+-----+--------------------+----+--------------------+--------+---------+
|UETunq2HsmwW9pbsS...|2014-03-02|    0|Yep, get the big ...| tip|VexAJmqOClKzm8KbY...|    2014|        3|
|UETunq2HsmwW9pbsS...|2014-03-02|    0|With your combo, ...| tip|VexAJmqOClKzm8KbY...|    2014|        3|
|UETunq2HsmwW9pbsS...|2014-07-27|    0|Traditional fish ...| tip|V8VL7b1J5WOLrZrTE...|    2014|        7|
|UETunq2HsmwW9pbsS...|2012-09-22|    0|Custom noodle style!| tip|e9HWnHUYlt9758oil...|    2012|        9|
|8Zlm1VmscYMn_9LtK...|2013-02-23|    0|They clean your g...| tip|jGuCy1xZnpIE0xppb...|    2013|        2|
|8Zlm1VmscYMn_9LtK...|2013-10-11|    0|     Shopping.. Love| tip|wRq6j5TozIiyPfVnZ...|    2013|       10|
|8Zlm1VmscYMn_9LtK...|2014-07-12|    0|Elegant

In [6]:
user_df.createOrReplaceTempView("user")

In [10]:
%%sql
select name,user_id,review_count from user order by review_count desc limit 5

+--------+--------------------+------------+
|    name|             user_id|review_count|
+--------+--------------------+------------+
|  Victor|JLM36sYWmouJAZ2kn...|        8062|
|Jennifer|22-6yC05pgWbLupHZ...|        4244|
|   Anita|pz97SxRe1Vk-5_K6E...|        3829|
|   Bruce|VhI6xyylcAxi0wOy2...|        3790|
| Kenneth|3zBKfA8-_fJRagWST...|        3337|
+--------+--------------------+------------+


In [21]:
%%sql 
select name,user_id, average_stars from user order by average_stars desc limit 10

+--------+--------------------+-------------+
|    name|             user_id|average_stars|
+--------+--------------------+-------------+
|    Ryan|Pg38ecKBtQl4ARKlu...|          5.0|
|     Liz|t4WMl2WTMF_SBPmFP...|          5.0|
| Susanna|0Hm4dzolMayQqYl1O...|          5.0|
|   Isaac|mh82Bh93fcpRcfR6C...|          5.0|
|   Frank|LK4Qu9zEcVn4G51I8...|          5.0|
|   Naeun|qBl3BXtZvA2Z_h8gG...|          5.0|
|  Nathan|DvhP9SV53SVlj8Hlv...|          5.0|
|Jennifer|5CqD_efRdiw_lDiTG...|          5.0|
|   Maria|NCy3OHjvsy8l1b-AF...|          5.0|
|     Eli|TMyTgSMmE2qQ2D6wF...|          5.0|
+--------+--------------------+-------------+


In [22]:
business_df.show()

+--------------------+--------------------+----------+------------+-----+-----+--------+
|         business_id|          categories|      city|review_count|stars|state|    type|
+--------------------+--------------------+----------+------------+-----+-----+--------+
|Hpxv1f3l99AHECGnJ...|[Hair Salons, Bea...|Enterprise|           9|  4.0|   NV|business|
|YCCDMLcb7UW8G-o_H...|[Steakhouses, Ame...|Scottsdale|         247|  4.0|   AZ|business|
|wdjehzTZcqSzYRKSy...|[Mexican, Restaur...| Las Vegas|          15|  4.5|   NV|business|
|QY0z7UW84ANFkJCYD...|    [Bakeries, Food]|   Phoenix|           3|  5.0|   AZ|business|
|yIuVbOjP6K7BRELWX...|[Delis, Restaurants]|Scottsdale|          68|  3.5|   AZ|business|
|Uv3dwiYJ1GXy-eb3o...| [Active Life, Golf]| Las Vegas|          23|  4.0|   NV|business|
|yAhVoAw0H0V77zGNJ...|[Food, Juice Bars...|  Paradise|          12|  3.0|   NV|business|
|qkbloHdDZuHf_0wTq...|       [Restaurants]|   Phoenix|         109|  3.5|   AZ|business|
|KBkyhge5PfKW4q-p1...

In [23]:
from pyspark.sql import Window
business_df_ctgy = business_df.groupBy("categories").agg(f.count("review_count").alias("Total_review_count"))
df_top_categories = business_df_ctgy.withColumn("rnk",f.row_number().over(Window.orderBy(f.col('Total_review_count').desc())))
df_top_categories = df_top_categories.filter(f.col('rnk')<=10)
df_top_categories.show()

+--------------------+------------------+---+
|          categories|Total_review_count|rnk|
+--------------------+------------------+---+
|[Mexican, Restaur...|              1230|  1|
|[Pizza, Restaurants]|               709|  2|
|[Beauty & Spas, N...|               681|  3|
|[Food, Coffee & Tea]|               665|  4|
|[Hotels & Travel,...|               632|  5|
|[Chinese, Restaur...|               616|  6|
|[Fast Food, Resta...|               518|  7|
|[Sandwiches, Rest...|               493|  8|
|[Hair Salons, Bea...|               444|  9|
|[American (Tradit...|               431| 10|
+--------------------+------------------+---+


In [32]:
business_df_clean = business_df.na.drop() #Dropping the rows having null values
business_df_one_category = business_df_clean.withColumn("categories",business_df_clean.categories[0]) #Selecting categories list first element in category
business_df_one_category.show()

+--------------------+--------------------+----------+------------+-----+-----+--------+
|         business_id|          categories|      city|review_count|stars|state|    type|
+--------------------+--------------------+----------+------------+-----+-----+--------+
|Hpxv1f3l99AHECGnJ...|         Hair Salons|Enterprise|           9|  4.0|   NV|business|
|YCCDMLcb7UW8G-o_H...|         Steakhouses|Scottsdale|         247|  4.0|   AZ|business|
|wdjehzTZcqSzYRKSy...|             Mexican| Las Vegas|          15|  4.5|   NV|business|
|QY0z7UW84ANFkJCYD...|            Bakeries|   Phoenix|           3|  5.0|   AZ|business|
|yIuVbOjP6K7BRELWX...|               Delis|Scottsdale|          68|  3.5|   AZ|business|
|Uv3dwiYJ1GXy-eb3o...|         Active Life| Las Vegas|          23|  4.0|   NV|business|
|yAhVoAw0H0V77zGNJ...|                Food|  Paradise|          12|  3.0|   NV|business|
|qkbloHdDZuHf_0wTq...|         Restaurants|   Phoenix|         109|  3.5|   AZ|business|
|KBkyhge5PfKW4q-p1...

In [33]:
business_df_ctgy = business_df_one_category.groupBy("categories").agg(f.count("review_count").alias("Total_review_count"))
df_top_categories = business_df_ctgy.withColumn("rnk",f.row_number().over(Window.orderBy(f.col('Total_review_count').desc())))
df_top_categories = df_top_categories.filter(f.col('rnk')<=10)
df_top_categories.show()

+--------------------+------------------+---+
|          categories|Total_review_count|rnk|
+--------------------+------------------+---+
|                Food|              4294|  1|
|            Shopping|              1885|  2|
|         Active Life|              1676|  3|
|                Bars|              1366|  4|
|      Local Services|              1351|  5|
|             Mexican|              1284|  6|
|     Hotels & Travel|              1283|  7|
|           Fast Food|               963|  8|
|Arts & Entertainment|               906|  9|
|         Hair Salons|               901| 10|
+--------------------+------------------+---+


In [34]:
business_df_one_category.createOrReplaceTempView("business")

In [35]:
%%sql
select categories,total_review_count from (
     select business_categories.*,
     ROW_NUMBER() OVER (ORDER BY total_review_count DESC) rn
     FROM (select categories,count(review_count) as total_review_count from business group by categories ) business_categories
     ) where rn <=10

+--------------------+------------------+
|          categories|total_review_count|
+--------------------+------------------+
|                Food|              4294|
|            Shopping|              1885|
|         Active Life|              1676|
|                Bars|              1366|
|      Local Services|              1351|
|             Mexican|              1284|
|     Hotels & Travel|              1283|
|           Fast Food|               963|
|Arts & Entertainment|               906|
|         Hair Salons|               901|
+--------------------+------------------+


In [37]:
business_review_df = business_df_one_category.groupBy("categories").agg(f.count("review_count").alias("total_review_count"))
top_business_df = business_review_df.filter(business_review_df.total_review_count >=1000).orderBy(f.desc("total_review_count"))
top_business_df.show()

+---------------+------------------+
|     categories|total_review_count|
+---------------+------------------+
|           Food|              4294|
|       Shopping|              1885|
|    Active Life|              1676|
|           Bars|              1366|
| Local Services|              1351|
|        Mexican|              1284|
|Hotels & Travel|              1283|
+---------------+------------------+


In [38]:
number_of_restaurants_df = business_df_one_category.select("state").groupBy("state").count().orderBy(f.desc("count"))
number_of_restaurants_df.show()

+-----+-----+
|state|count|
+-----+-----+
|   AZ|22181|
|   NV|14583|
|  EDH| 2841|
|   WI| 2118|
|   ON|  305|
|  MLN|  102|
|  ELN|    8|
|  FIF|    3|
|  SCB|    2|
|   NY|    2|
|   NC|    1|
|  KHL|    1|
|   MA|    1|
|   MN|    1|
|  NTH|    1|
|  XGL|    1|
|   CA|    1|
|   GA|    1|
+-----+-----+


In [42]:
%%sql
select *from (
    select state,city,review_count,
    ROW_NUMBER() OVER(PARTITION BY state order by review_count desc ) rn 
    from business 
    ) where rn<=3

+-----+---------+------------+---+
|state|     city|review_count| rn|
+-----+---------+------------+---+
|   AZ|  Phoenix|        1325|  1|
|   AZ|  Phoenix|        1289|  2|
|   AZ|    Tempe|        1110|  3|
|   GA|  Atlanta|          16|  1|
|  XGL|Edinburgh|           4|  1|
|   MA|Middleton|           5|  1|
|   MN|    Eagan|           3|  1|
|   NY|Rochester|          10|  1|
|   NY| New York|           4|  2|
|   ON| Waterloo|          48|  1|
|   ON| Waterloo|          39|  2|
|   ON| Waterloo|          38|  3|
|   NV|Las Vegas|        4084|  1|
|   NV|Las Vegas|        3655|  2|
|   NV|Las Vegas|        3408|  3|
|  EDH|Edinburgh|         239|  1|
|  EDH|Edinburgh|         117|  2|
|  EDH|Edinburgh|         112|  3|
|   WI|  Madison|         619|  1|
|   WI|  Madison|         406|  2|
+-----+---------+------------+---+
only showing top 20 rows


In [55]:
#Top 5 cities in NV where people reviewed the most 
business_nv_df =  business_df_one_category.filter(business_df_one_category.state=="NV")
top_ctgy_nv_df = business_nv_df.groupBy('city').agg(f.count("review_count").alias("total_review_count"))
top_ctgy_nv_df = top_ctgy_nv_df.orderBy(f.desc('total_review_count')).limit(5)
top_ctgy_nv_df.show()

+---------------+------------------+
|           city|total_review_count|
+---------------+------------------+
|      Las Vegas|             12021|
|      Henderson|              1914|
|North Las Vegas|               521|
|   Boulder City|                24|
|       Paradise|                20|
+---------------+------------------+


In [56]:
tip_df.show()

+--------------------+----------+-----+--------------------+----+--------------------+--------+---------+
|         business_id|      date|likes|                text|type|             user_id|tip_year|tip_month|
+--------------------+----------+-----+--------------------+----+--------------------+--------+---------+
|UETunq2HsmwW9pbsS...|2014-03-02|    0|Yep, get the big ...| tip|VexAJmqOClKzm8KbY...|    2014|        3|
|UETunq2HsmwW9pbsS...|2014-03-02|    0|With your combo, ...| tip|VexAJmqOClKzm8KbY...|    2014|        3|
|UETunq2HsmwW9pbsS...|2014-07-27|    0|Traditional fish ...| tip|V8VL7b1J5WOLrZrTE...|    2014|        7|
|UETunq2HsmwW9pbsS...|2012-09-22|    0|Custom noodle style!| tip|e9HWnHUYlt9758oil...|    2012|        9|
|8Zlm1VmscYMn_9LtK...|2013-02-23|    0|They clean your g...| tip|jGuCy1xZnpIE0xppb...|    2013|        2|
|8Zlm1VmscYMn_9LtK...|2013-10-11|    0|     Shopping.. Love| tip|wRq6j5TozIiyPfVnZ...|    2013|       10|
|8Zlm1VmscYMn_9LtK...|2014-07-12|    0|Elegant

In [57]:
user_df.show()

+-------------+-------+------------+--------------------+-------------+
|average_stars|   name|review_count|             user_id|yelping_since|
+-------------+-------+------------+--------------------+-------------+
|         4.07|  Suzzi|          15|WWoIIo1OcPWW3LFTR...|      2012-10|
|         3.33|  Chris|           3|ssaX6cEzE09MTMR5G...|      2009-12|
|         2.75|  Derek|          16|Wvtn9aKFXGcIeKaak...|      2011-02|
|         3.69|  Erika|          26|RZxdXUG9oUQKje9ZZ...|      2012-07|
|         3.32|  Tommy|         333|wXyJSZYXgEgzQZg7q...|      2007-04|
|          3.0|  Jenny|           2|cb_nRxYeu3EDEWksv...|      2011-09|
|          3.8|    Joe|          21|Y8dD8_IZ3zSGHirfD...|      2009-12|
|          2.0|Johanna|          10|gIqWui2rv_z1CqQC9...|      2011-07|
|         3.33|   Hoss|           6|i_O2jcugpA-vLi1Ou...|      2011-04|
|          5.0|  Maria|           2|DimqVMv_ob4qq1r3s...|      2013-04|
|         3.57| Zennea|           7|2qdVsSxliXh08zdRd...|      2

In [58]:
tip_user_df = tip_df.join(user_df,"user_id","inner")
tip_user_df.show()

+--------------------+--------------------+----------+-----+--------------------+----+--------+---------+-------------+--------+------------+-------------+
|             user_id|         business_id|      date|likes|                text|type|tip_year|tip_month|average_stars|    name|review_count|yelping_since|
+--------------------+--------------------+----------+-----+--------------------+----+--------+---------+-------------+--------+------------+-------------+
|VexAJmqOClKzm8KbY...|UETunq2HsmwW9pbsS...|2014-03-02|    0|Yep, get the big ...| tip|    2014|        3|         3.62|     Kay|         159|      2012-11|
|VexAJmqOClKzm8KbY...|UETunq2HsmwW9pbsS...|2014-03-02|    0|With your combo, ...| tip|    2014|        3|         3.62|     Kay|         159|      2012-11|
|V8VL7b1J5WOLrZrTE...|UETunq2HsmwW9pbsS...|2014-07-27|    0|Traditional fish ...| tip|    2014|        7|         3.91|     Jim|          11|      2011-10|
|e9HWnHUYlt9758oil...|UETunq2HsmwW9pbsS...|2012-09-22|    0|Cust

In [59]:
tip_user_df.count()

403210


In [62]:
#Most tipped year in desc order
most_tipped_year = tip_user_df.groupBy("tip_year").count().orderBy(f.desc('count'))
most_tipped_year.show()

+--------+------+
|tip_year| count|
+--------+------+
|    2012|118777|
|    2011| 97329|
|    2013| 95870|
|    2014| 63435|
|    2010| 27301|
|    2009|   498|
+--------+------+


In [63]:
#Month , year most tipped in desc order
most_tipped_month_year = tip_user_df.groupBy("tip_year","tip_month").count().orderBy(f.desc('count'))
most_tipped_month_year.show()

+--------+---------+-----+
|tip_year|tip_month|count|
+--------+---------+-----+
|    2012|        7|12983|
|    2012|        6|12644|
|    2012|        8|12415|
|    2012|        5|12060|
|    2012|        3|11773|
|    2012|        4|11159|
|    2014|        3|10553|
|    2014|        5|10251|
|    2011|        8|10155|
|    2012|        1| 9895|
|    2014|        4| 9562|
|    2011|        7| 9511|
|    2012|        2| 9497|
|    2011|       10| 9266|
|    2011|        9| 9252|
|    2011|       12| 9226|
|    2013|        8| 9177|
|    2011|       11| 8894|
|    2014|        6| 8867|
|    2013|        3| 8831|
+--------+---------+-----+
only showing top 20 rows


In [ ]:
#User who gave the maximum number of tips
user_max_tip_df = tip_user_df.group("user_id").agg(f.count("review_count").alias("total_review_count"))
user_max_tip_df = user_max_tip_df.orderBy(f.desc('total_review_count'))
user_max_tip_df.show()